# BEE 4750: Final Project

Members: Jonathan Marcuse (jrm564), Geovany Rodriguez (ga283), Oliver Sandiford (ots22)

In [19]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

  Activating project at `~/Desktop/Final Projects/4750_Project`


In [20]:
#initiating several packages that may be useful later
Pkg.add("Plots")
Pkg.add("CSV")
Pkg.add("DataFrames")
Pkg.add("Dates")
using Plots
using CSV
using DataFrames
using Dates

   Resolving package versions...


  No Changes to `~/Desktop/Final Projects/4750_Project/Project.toml`
  No Changes to `~/Desktop/Final Projects/4750_Project/Manifest.toml`


   Resolving package versions...


  No Changes to `~/Desktop/Final Projects/4750_Project/Project.toml`
  No Changes to `~/Desktop/Final Projects/4750_Project/Manifest.toml`


   Resolving package versions...


  No Changes to `~/Desktop/Final Projects/4750_Project/Project.toml`
  No Changes to `~/Desktop/Final Projects/4750_Project/Manifest.toml`


   Resolving package versions...


  No Changes to `~/Desktop/Final Projects/4750_Project/Project.toml`
  No Changes to `~/Desktop/Final Projects/4750_Project/Manifest.toml`


#### Implementing the first component of the model - Determining how many aircrafts can land per hour

In [21]:
#Initialize flexible variables
aircraft_landing_speeds = [130 137.5 145 152.5 160]; #landing speeds of each type of aircraft in knots (interpolated)
proportions = [0.101333333 0.248 0.242666667 0.370666667 0.037333333]; #proportion of each type of aircraft, corresponding to the number of ones listed above
delta = 3; #minimum spacing between aircrafts in nautical miles (safety parameter) - taken from CEE 4640 safety parameter
gamma = [(11000/6076.11549) (10000/6076.11549) (6800/6076.11549)]; #length of the runway in nautical miles for each runway
#converted length in ft to nautical miles with 6,076.11549 ft per nautical mile

1×3 Matrix{Float64}:
 1.81037  1.64579  1.11914

In [25]:
for k in 1:3
    l = length(aircraft_landing_speeds);
    Time_Space = zeros(l,l);
    Time_Probabilities = zeros(l,l);
    for i in 1:l
        for j in 1:l
            Time_Space[i,j] = aircraft_landing_speeds[i] > aircraft_landing_speeds[j] ? ((delta/aircraft_landing_speeds[i])*3600) : (((delta/aircraft_landing_speeds[i])+(gamma[k]/aircraft_landing_speeds[i])-(gamma[k]/aircraft_landing_speeds[j]))*3600)
            Time_Probabilities[i,j] = proportions[i]*proportions[j]*Time_Space[i,j];
        end
    end
    avg_time = sum(Time_Probabilities); #total average seconds between aircraft landings
    aircrafts_per_hour = floor((1/avg_time)*3600); #floor functin used because you cant have a fraction of an aircraft land
    println("The average amount of time in between aircrafts each hour on runway $k is $avg_time seconds")
    println("The total number of aircrafts that can land per hour on average on runway $k is $aircrafts_per_hour aircrafts/hour")
    println(" ")
end

The average amount of time in between aircrafts each hour on runway 1 is 76.15490471682274 seconds
The total number of aircrafts that can land per hour on average on runway 1 is 47.0 aircrafts/hour
 
The average amount of time in between aircrafts each hour on runway 2 is 76.02652327219118 seconds
The total number of aircrafts that can land per hour on average on runway 2 is 47.0 aircrafts/hour
 
The average amount of time in between aircrafts each hour on runway 3 is 75.61570264937018 seconds
The total number of aircrafts that can land per hour on average on runway 3 is 47.0 aircrafts/hour
 


The first matrix above is the needed spacing between aircrafts landing in seconds, where the first row and first column are the speed of the first aircraft listed in the aircraft_landing_speeds vector. The second matrix above is the proportion of each type of aircraft multiplied by each other and then by the time spacing to form the time probability matrix that will later be used to find the total average time between aircrafts by summing all of its elements.

---

- historical runway usage --> flight demand data
- weather conditions --> what weather conditions cause flight to be cancelled
- Air traffic volume & air wake concerns--> model already built out
- passenger load and unload time

Average number of days per month with wind speeds above 38mph:
January had 1.7 days above 38mph, 1.7 feb, 1.6 days in march, 0.2 april, 0 may, 0 june, 0 july, 0 august, 0 sept, 0.1 oct, 0.4 nov, 1.9 dec

---

### Load in Wind Speed Data
This has the wind speeds at Newark for every 3 minutes of the month of August in 2023. This will be used to tell us how much to delay flights by given the wind speed. Based on our background research, if the wind speed is between 30mph and 40pmh, the aircraft will be delayed by 30 minutes after the latest instance of winds in that speed. If the wind speed is greater than 40mph, the aircraft will be delayed by 1 hour from the latest instance of that wind speed. If the wind speed is under 30mph it is clear for takeoff.

In [23]:
# Load the CSV file into a DataFrame
Wind_DataFrame = DataFrame(CSV.File("Wind_Data.csv"));

# Extract wind speed and date columns
wind_speed = Wind_DataFrame."Wind Speed (mph)";
date_string = Wind_DataFrame."Date";

# Parse the date string with the correct format
date_value = Dates.DateTime.(date_string, "mm/dd/yyyy HH:MM");